# Advanced models in `pypsa`: capacity expansion planning with sector coupling


:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa pandas numpy matplotlib plotly highspy
```
:::

## Problem description

To explore capacity expansion problem with sector-coupling options, let's model an "greenfield" energy system model with:
- electricity demand (here: Germany 2015)
- several technologies for electricity generation (wind, solar, and gas for peak load)
- hydrogen consumption (e.g., an offtaker in the industrial sector)
- hydrogen production from electrolysis
- hydrogen storage
- hydrogen fuel cell

In [ ]:
import pypsa
import pandas as pd

pd.options.plotting.backend = "plotly"

## Prepare technology data

At TU Berlin, we maintain a database (https://github.com/PyPSA/technology-data) which collects assumptions and projections for energy system technologies (such as costs, efficiencies, lifetimes, etc.) for given years, which we use for our research.

Reading this data into a useable `pandas.DataFrame` requires some pre-processing (e.g. converting units, setting defaults, re-arranging dimensions):

In [ ]:
YEAR = 2030
url = f"https://raw.githubusercontent.com/PyPSA/technology-data/master/outputs/costs_{YEAR}.csv"
costs = pd.read_csv(url, index_col=[0, 1])

In [ ]:
costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
costs.unit = costs.unit.str.replace("/kW", "/MW")

defaults = {
    "FOM": 0,
    "VOM": 0,
    "efficiency": 1,
    "fuel": 0,
    "investment": 0,
    "lifetime": 25,
    "CO2 intensity": 0,
    "discount rate": 0.07,
}
costs = costs.value.unstack().fillna(defaults)

costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
# costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]
costs.at["OCGT", "CO2 intensity"] = costs.at["gas", "CO2 intensity"]
# costs.at["CCGT", "CO2 intensity"] = costs.at["gas", "CO2 intensity"]

In [ ]:
# costs

Let's also write a small utility _function_ that calculates the **annuity** to annualise investment costs. The formula is

$$
a(r, n) = \frac{r}{1-(1+r)^{-n}}
$$
where $r$ is the discount rate and $n$ is the lifetime.

In [ ]:
def annuity_factor(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

The resulting `annuity_factor` (often called capital recovery factor) represents the factor that, when multiplied by the initial capital cost, yields the equivalent annual cost accounting for the time value of money:

In [ ]:
annuity_factor(0.07, 20)

Based on this, we can calculate the short-term marginal generation costs (€/MWh)

In [ ]:
costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

and the annualised investment costs (`capital_cost` in PyPSA terms, €/MW/a):

In [ ]:
annuity = costs.apply(lambda x: annuity_factor(0.07, x["lifetime"]), axis=1)

In [ ]:
costs["capital_cost"] = (annuity + costs["FOM"] / 100) * costs["investment"]

## Prepare load and renewable generation time-series

We are also going to need some time series for wind, solar and load.

For now, we are going to recycle the time series we used in the `pandas` tutorial.

In [ ]:
url = (
    "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
)
ts = pd.read_csv(url, index_col=0, parse_dates=True)

# fallback if url is not available
# ts = pd.read_csv("../data/time-series-lecture-2.csv", index_col=0, parse_dates=True)

In [ ]:
ts.head()

Let's also convert the load time series from GW to MW, the base unit of PyPSA:

In [ ]:
ts.load *= 1e3

Optionally, we can downscale temporal resolution of the time series to save some computation time:

In [ ]:
# here we sample only every other hour:
resolution = 4
ts = ts.resample(f"{resolution}h").first()

## Build our energy system to simulate

As always, let's initialize an empty network:

In [ ]:
n = pypsa.Network()

Then, we add a single electricity bus...

In [ ]:
n.add("Bus", "electricity")

...and tell the `pypsa.Network` object `n` what the snapshots of the model will be using the utility function `n.set_snapshots()`.

In [ ]:
n.set_snapshots(ts.index)

In [ ]:
n.snapshots

If we resampled the time series above, we need to adjust the weighting of the snapshots (i.e. how many hours they represent). We can do that with `n.snapshot_weightings`:

In [ ]:
n.snapshot_weightings.head(3)

In [ ]:
n.snapshot_weightings.loc[:, :] = resolution

In [ ]:
n.snapshot_weightings.head(3)

### Adding components for electricity part

Then, we add all the technologies we are going to include as carriers.

In [ ]:
carriers = [
    "onwind",
    "offwind",
    "solar",
    "OCGT",
    "hydrogen storage underground",
]

n.add(
    "Carrier",
    carriers,
    color=["dodgerblue", "aquamarine", "gold", "indianred", "magenta"],
    co2_emissions=[costs.at[c, "CO2 intensity"] for c in carriers],
)

Next, we add the demand time series to the model.

In [ ]:
n.add(
    "Load",
    "demand",
    bus="electricity",
    p_set=ts.load,
)

Let's have a check whether the data was read-in correctly.

In [ ]:
n.loads_t.p_set.plot()

We are going to add one dispatchable generation technology to the model. This is an open-cycle gas turbine (OCGT) with CO$_2$ emissions of 0.2 t/MWh$_{th}$.

In [ ]:
n.add(
    "Generator",
    "OCGT",
    bus="electricity",
    carrier="OCGT",
    capital_cost=costs.at["OCGT", "capital_cost"],
    marginal_cost=costs.at["OCGT", "marginal_cost"],
    efficiency=costs.at["OCGT", "efficiency"],
    p_nom_extendable=True,
)

Adding the variable renewable generators works almost identically, but we also need to supply the capacity factors to the model via the attribute `p_max_pu`.

In [ ]:
for tech in ["onwind", "offwind", "solar"]:
    n.add(
        "Generator",
        tech,
        bus="electricity",
        carrier=tech,
        p_max_pu=ts[tech],
        capital_cost=costs.at[tech, "capital_cost"],
        marginal_cost=costs.at[tech, "marginal_cost"],
        efficiency=costs.at[tech, "efficiency"],
        p_nom_extendable=True,
    )

In [ ]:
# Making sure the capacity factors are read-in correctly
n.generators_t.p_max_pu.loc["2015-03"].plot()

### Adding components for hydrogen part

Add a dedicated `Bus` for the hydrogen energy carrier:

In [ ]:
n.add("Bus", "hydrogen", carrier="hydrogen")

Add a `Link` for the hydrogen electrolysis:

In [ ]:
n.add(
    "Link",
    "electrolysis",
    bus0="electricity",
    bus1="hydrogen",
    carrier="electrolysis",
    p_nom_extendable=True,
    efficiency=0.7,
    capital_cost=50e3,  # €/MW/a
)

:::{note}
Some of the sector-coupling technologies we are going to add have multiple ouputs (e.g. CHP plants producing heat and power). PyPSA can automatically handle links have more than one input (`bus0`)
and/or output (i.e. `bus1`, `bus2`, `bus3`) with a given efficieny (`efficiency`, `efficiency2`, `efficiency3`).
:::

Add a `Link` for the fuel cell which reconverts hydrogen to electricity:

In [ ]:
n.add(
    "Link",
    "fuel cell",
    bus0="hydrogen",
    bus1="electricity",
    carrier="fuel cell",
    p_nom_extendable=True,
    efficiency=0.5,
    capital_cost=120e3,  # €/MW/a
)

Add a `Store` for the hydrogen storage:

In [ ]:
n.add(
    "Store",
    "hydrogen storage",
    bus="hydrogen",
    carrier="hydrogen storage",
    capital_cost=140,  # €/MWh/a
    e_nom_extendable=True,
    e_cyclic=True,  # cyclic state of charge
)

To model an industrial hydrogen offtaker, we add also a hydrogen demand to the hydrogen bus.

In the example below, we add a hydrogen demand such that it equals ~25% of the electricity demand (in MWh_H2)



In [ ]:
n.add(
    "Load", "hydrogen demand", bus="hydrogen", carrier="hydrogen", p_set=19500
)  # MWh_H2/h

### We are now ready to solve the model 

In [ ]:
# n.optimize.create_model()

In [ ]:
n.optimize(solver_name="highs")
# 66 seconds for 1H temporal resolution (8760 snapshots)
# 7 seconds for 4H temporal resolution (2190 snapshots)

### Exploring model results

The total system cost in billion Euros per year:

In [ ]:
n.objective / 1e9

`n.statistics()` provides an informative overview of the model results.

See documentation: https://pypsa.readthedocs.io/en/stable/api/statistics.html

In [ ]:
n.statistics()

We can use `n.statisitcs()` to get a quick overview of optimised capacities across all components:

In [ ]:
n.statistics.expanded_capacity().div(1e3).round(1)  # GW

You can also use `n.statistics()` to promptly get an energy balance for the complete system or even any specific bus:

In [ ]:
n.statistics.energy_balance(aggregate_time=False)

In [ ]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier="AC").div(1e3).groupby(
    "carrier"
).sum().T.plot()

In [ ]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier="hydrogen").div(
    1e3
).groupby("carrier").sum().T.plot()

Possibly, we are also interested in the total emissions:

In [ ]:
emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)  # t/h

In [ ]:
n.snapshot_weightings.generators @ emissions.sum(axis=1).div(1e6)  # Mt

## Adding emission limits

The gas power plant offers sufficient and cheap enough backup capacity to run in periods of low wind and solar generation. But what happens if this source of flexibility disappears. Let's model a 100% renewable electricity system by adding a CO$_2$ emission limit as global constraint:

In [ ]:
n.add(
    "GlobalConstraint",
    "CO2Limit",
    carrier_attribute="co2_emissions",
    sense="<=",
    constant=0,
)

When we run the model now...

In [ ]:
n.optimize(solver_name="highs")

The total system cost (billion Euros per year) is now higher than before:

In [ ]:
n.objective / 1e9

The optimal capacity mix now does not include any gas power plants and includes onshore wind and fuel cell technologies:

In [ ]:
n.statistics.expanded_capacity().div(1e3).round(1)  # GW

Fuel cell technology steps in hours with low wind and solar generation:

In [ ]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier="AC").div(1e3).groupby(
    "carrier"
).sum().T.plot()

In [ ]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier="hydrogen").div(
    1e3
).groupby("carrier").sum().T.plot()

In [ ]:
n.stores_t.e.plot()

Total emissions are now zero:

In [ ]:
emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)  # t/h

In [ ]:
n.snapshot_weightings.generators @ emissions.sum(axis=1).div(1e6)  # Mt